In [2]:
# 1. 필요한 라이브러리 불러오기
import pandas as pd

# 2. Parquet 파일 읽어서 변수에 저장하기
# 'your_file.parquet' 부분은 실제 파일 이름으로 바꿔주세요.
df_train_y = pd.read_parquet("C:/Users/song2/Downloads/jj/train_y_yeongduk.parquet")

# 3. 데이터가 잘 읽혔는지 확인하기 (상위 5개 행만 미리보기)
df_train_y.head()



,end_datetime,구분,시간,energy_mwh,energy_kwh
720,2024-05-01 00:00:00+09:00,2024-04-30,24,10.321,10321.0
721,2024-05-01 01:00:00+09:00,2024-05-01,1,8.475,8475.0
722,2024-05-01 02:00:00+09:00,2024-05-01,2,12.780,12780.0
723,2024-05-01 03:00:00+09:00,2024-05-01,3,19.972,19972.0
724,2024-05-01 04:00:00+09:00,2024-05-01,4,17.984,17984.0


In [3]:
# 2. Parquet 파일 읽어서 변수에 저장하기
# 'your_file.parquet' 부분은 실제 파일 이름으로 바꿔주세요.
df_scada = pd.read_parquet("C:/Users/song2/Downloads/jj/scada_yeongduk.parquet")

# 3. 데이터가 잘 읽혔는지 확인하기 (상위 5개 행만 미리보기)
df_scada.head()

,dt,wind_speed_mps,wind_direction_degree,turbine_id
0,2024-04-04 00:00:00+09:00,3.36,95.298518,WTG01
1,2024-04-04 00:00:00+09:00,4.38,NaN,WTG02
2,2024-04-04 00:00:00+09:00,4.30,NaN,WTG03
3,2024-04-04 00:00:00+09:00,3.90,NaN,WTG04
4,2024-04-04 00:00:00+09:00,NaN,NaN,WTG05


In [4]:
# ==============================================================================
# 0. 라이브러리 불러오기
# ==============================================================================
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings

# 불필요한 경고 메시지 무시
warnings.filterwarnings('ignore')
print("라이브러리 로딩 완료")

라이브러리 로딩 완료


In [5]:
#영덕풍력




In [6]:
# 1. 데이터 불러오기 및 기본 전처리
 #영덕풍력의 발전량 및 SCADA(센서) 데이터를 불러온 후, 날짜 형식을 변환하고 풍속의 극단적인 값을 일부 조정

try:
    df_train_y_yd = pd.read_parquet("C:/Users/song2/Downloads/jj/train_y_yeongduk.parquet")
    df_scada_yd = pd.read_parquet("C:/Users/song2/Downloads/jj/scada_yeongduk.parquet")
    print("\n[영덕] 데이터 파일 로딩 완료")
except FileNotFoundError:
    print("[영덕] 오류: Parquet 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    df_train_y_yd, df_scada_yd = pd.DataFrame(), pd.DataFrame()

if not df_train_y_yd.empty and not df_scada_yd.empty:
    df_train_y_yd['end_datetime'] = pd.to_datetime(df_train_y_yd['end_datetime'])
    df_scada_yd['dt'] = pd.to_datetime(df_scada_yd['dt'])
    df_scada_yd.columns = df_scada_yd.columns.str.strip()
    lower_bound = df_scada_yd['wind_speed_mps'].quantile(0.01)
    upper_bound = df_scada_yd['wind_speed_mps'].quantile(0.99)
    df_scada_yd['wind_speed_mps'] = df_scada_yd['wind_speed_mps'].clip(lower_bound, upper_bound)
    print("[영덕] 데이터 기본 전처리 완료")


[영덕] 데이터 파일 로딩 완료
[영덕] 데이터 기본 전처리 완료


In [ ]:
# 2. 피처 엔지니어링 (SCADA 데이터 가공)
 # SCADA 데이터로부터 월, 시간, 요일 등 시간 관련 특성을 만들고, 과거 풍속(Lag), 이동 평균(Rolling) 등 모델 학습에 유용한 여러 특성들을 생성

if not df_scada_yd.empty:
    #시간 기반 특성 생성
    df_scada_yd['month'] = df_scada_yd['dt'].dt.month
    df_scada_yd['hour'] = df_scada_yd['dt'].dt.hour
    df_scada_yd['dayofweek'] = df_scada_yd['dt'].dt.dayofweek
    df_scada_yd['is_weekend'] = df_scada_yd['dayofweek'].isin([5, 6]).astype(int)
    #주기성 데이터 변환
    df_scada_yd['hour_sin'] = np.sin(2 * np.pi * df_scada_yd['hour'] / 24)
    df_scada_yd['hour_cos'] = np.cos(2 * np.pi * df_scada_yd['hour'] / 24)
    df_scada_yd['wind_dir_sin'] = np.sin(np.deg2rad(df_scada_yd['wind_direction_degree']))
    df_scada_yd['wind_dir_cos'] = np.cos(np.deg2rad(df_scada_yd['wind_direction_degree']))
    #시계열 특성 생성
    df_scada_yd = df_scada_yd.sort_values(by=['turbine_id', 'dt'])
    for lag in [1, 24]:
        df_scada_yd[f'wind_speed_mps_lag{lag}'] = df_scada_yd.groupby('turbine_id')['wind_speed_mps'].shift(lag)
    rolling_24h = df_scada_yd.groupby('turbine_id')['wind_speed_mps'].rolling(window=24, min_periods=1)
    # 최근 24시간 동안의 풍속 추세(Rolling)를 파악하기 위해 이동 평균, 표준편차, 최대/최소값
    df_scada_yd['roll_mean_24h'] = rolling_24h.mean().reset_index(level=0, drop=True)
    df_scada_yd['roll_std_24h'] = rolling_24h.std().reset_index(level=0, drop=True)
    df_scada_yd['roll_max_24h'] = rolling_24h.max().reset_index(level=0, drop=True)
    df_scada_yd['roll_min_24h'] = rolling_24h.min().reset_index(level=0, drop=True)
    # 특정 시간대와 풍속의 복합적인 관계를 나타내는 상호작용 특성
    df_scada_yd['wind_x_hour_sin'] = df_scada_yd['wind_speed_mps'] * df_scada_yd['hour_sin']
    df_scada_yd = pd.get_dummies(df_scada_yd, columns=['turbine_id'], prefix='turbine')
    df_scada_yd = df_scada_yd.interpolate(method='linear').bfill().ffill()
    print("[영덕] 피처 엔지니어링 완료")

[영덕] 피처 엔지니어링 완료


In [8]:
# 3. 최종 학습 데이터 생성
 # 전처리된 SCADA 데이터와 발전량 데이터를 시간 기준으로 병합하고, 영덕 모델은 홀수 달 데이터만 학습에 사용하도록 분리

if not df_train_y_yd.empty and not df_scada_yd.empty:
    full_data_yd = pd.merge_asof(
        df_train_y_yd.sort_values("end_datetime"),
        df_scada_yd.sort_values("dt"),
        left_on="end_datetime",
        right_on="dt",
        direction="backward"
    )
    full_data_yd['month'] = full_data_yd['end_datetime'].dt.month
    train_data_yd = full_data_yd[full_data_yd['month'] % 2 != 0].copy()
    features_yd = [col for col in train_data_yd.columns if col not in [
        'end_datetime', 'dt', 'energy_mwh', '구분', '시간', 'energy_kwh',
        'month', 'hour', 'dayofweek'
    ]]
    X_yd = train_data_yd[features_yd]
    y_yd = train_data_yd['energy_kwh']
    print("[영덕] 최종 학습 데이터셋 생성 완료")


[영덕] 최종 학습 데이터셋 생성 완료


In [9]:
train_data_yd 

,end_datetime,구분,시간,energy_mwh,energy_kwh,dt,wind_speed_mps,wind_direction_degree,month,hour,...,roll_min_24h,wind_x_hour_sin,turbine_WTG01,turbine_WTG02,turbine_WTG03,turbine_WTG04,turbine_WTG05,turbine_WTG06,turbine_WTG07,turbine_WTG08
0,2024-05-01 00:00:00+09:00,2024-04-30,24,10.321,10321.0,2024-05-01 00:00:00+09:00,8.559999,-145.905442,5,0,...,8.099999,0.000000,False,False,False,False,False,False,True,False
1,2024-05-01 01:00:00+09:00,2024-05-01,1,8.475,8475.0,2024-05-01 01:00:00+09:00,3.830000,76.422776,5,1,...,3.700000,0.991277,True,False,False,False,False,False,False,False
2,2024-05-01 02:00:00+09:00,2024-05-01,2,12.780,12780.0,2024-05-01 02:00:00+09:00,8.530000,-145.447373,5,2,...,4.210000,4.265000,False,True,False,False,False,False,False,False
3,2024-05-01 03:00:00+09:00,2024-05-01,3,19.972,19972.0,2024-05-01 03:00:00+09:00,8.150000,-139.178641,5,3,...,6.330000,5.762920,False,False,True,False,False,False,False,False
4,2024-05-01 04:00:00+09:00,2024-05-01,4,17.984,17984.0,2024-05-01 04:00:00+09:00,5.170000,75.922447,5,4,...,3.700000,4.477351,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,2025-03-31 19:00:00+09:00,2025-03-31,19,4.325,4325.0,2025-03-31 19:00:00+09:00,5.502613,176.229679,3,19,...,5.502524,-3.833538,False,False,False,True,False,False,False,False
4412,2025-03-31 20:00:00+09:00,2025-03-31,20,2.515,2515.0,2025-03-31 20:00:00+09:00,3.621124,-176.902505,3,20,...,3.621234,-0.010476,False,False,False,False,False,True,False,False
4413,2025-03-31 21:00:00+09:00,2025-03-31,21,3.320,3320.0,2025-03-31 21:00:00+09:00,4.311409,-169.556820,3,21,...,4.311468,-0.007439,False,True,False,False,False,False,False,False
4414,2025-03-31 22:00:00+09:00,2025-03-31,22,3.012,3012.0,2025-03-31 22:00:00+09:00,7.620000,-160.098089,3,22,...,7.620000,-1.972201,False,False,False,False,False,False,False,True


In [10]:
# 4. 최종 모델 학습 (영덕)
 # 하이퍼파라미터 튜닝: Optuna와 TimeSeriesSplit(n_splits=5)를 사용하여 교차 검증을 수행함
 # RMSE를 기준으로 최적의 파라미터 조합을 탐색, 미리 찾아낸 최적의 파라미터를 사용하여 XGBoost 모델을 학습

best_params_yd = {
    'lambda': 0.08972182871699765, 
    'alpha': 0.6417387443188353, 
    'colsample_bytree': 0.8774368315279678,
    'subsample': 0.7680660705013798, 
    'learning_rate': 0.025813031282491863,
    'n_estimators': 314,
    'max_depth': 6, 
    'min_child_weight': 33
}
final_model_yd = xgb.XGBRegressor(random_state=42, n_jobs=-1, **best_params_yd)
final_model_yd.fit(X_yd, y_yd)
print("[영덕] 최적 파라미터로 최종 모델 학습 완료")

[영덕] 최적 파라미터로 최종 모델 학습 완료


In [11]:
# 5. 미래 데이터 생성 및 예측 (영덕)
 # 학습된 모델을 사용하여 예측 대상 기간인 짝수 달의 발전량을 예측
 # 미래의 날씨는 과거 데이터의 월/시간별 평균값을 사용

future_timestamps_yd = pd.to_datetime(pd.date_range(start='2024-04-01', end='2025-03-31', freq='h'))
future_timestamps_yd = future_timestamps_yd[future_timestamps_yd.month.isin([2, 4, 6, 8, 10, 12])]
all_turbines_yd = [col for col in full_data_yd.columns if col.startswith('turbine_WTG')]
X_future_base_yd = pd.DataFrame({'dt': future_timestamps_yd})

# 과거 짝수달의 월/시간별 평균 SCADA 데이터 생성
even_month_scada_yd = df_scada_yd[df_scada_yd['month'].isin([2, 4, 6, 8, 10, 12])]
historical_averages_yd = even_month_scada_yd.groupby(['month', 'hour']).agg(
    wind_speed_mps=('wind_speed_mps', 'mean'), wind_direction_degree=('wind_direction_degree', 'mean')
).reset_index()
X_future_base_yd['month'] = X_future_base_yd['dt'].dt.month
X_future_base_yd['hour'] = X_future_base_yd['dt'].dt.hour
X_future_base_yd = pd.merge(X_future_base_yd, historical_averages_yd, on=['month', 'hour'], how='left')
X_future_base_yd['dayofweek'] = X_future_base_yd['dt'].dt.dayofweek
X_future_base_yd['is_weekend'] = X_future_base_yd['dayofweek'].isin([5, 6]).astype(int)
X_future_base_yd['hour_sin'] = np.sin(2 * np.pi * X_future_base_yd['hour'] / 24)
X_future_base_yd['hour_cos'] = np.cos(2 * np.pi * X_future_base_yd['hour'] / 24)
X_future_base_yd['wind_dir_sin'] = np.sin(np.deg2rad(X_future_base_yd['wind_direction_degree']))
X_future_base_yd['wind_dir_cos'] = np.cos(np.deg2rad(X_future_base_yd['wind_direction_degree']))
X_future_base_yd['wind_speed_mps_lag1'] = X_future_base_yd['wind_speed_mps']
X_future_base_yd['wind_speed_mps_lag24'] = X_future_base_yd['wind_speed_mps']
X_future_base_yd['roll_mean_24h'] = X_future_base_yd['wind_speed_mps']
X_future_base_yd['roll_std_24h'] = 0
X_future_base_yd['roll_max_24h'] = X_future_base_yd['wind_speed_mps']
X_future_base_yd['roll_min_24h'] = X_future_base_yd['wind_speed_mps']
X_future_base_yd['wind_x_hour_sin'] = X_future_base_yd['wind_speed_mps'] * X_future_base_yd['hour_sin']
future_pred_list_yd = []
for turbine_col in all_turbines_yd:
    temp_df = X_future_base_yd.copy()
    for col in all_turbines_yd:
        temp_df[col] = (col == turbine_col)
    future_pred_list_yd.append(temp_df)
X_future_final_yd = pd.concat(future_pred_list_yd).sort_values('dt').reset_index(drop=True)
X_future_final_yd = X_future_final_yd.interpolate(method='linear').bfill().ffill()
X_future_final_yd = X_future_final_yd[features_yd]
future_predictions_yd = final_model_yd.predict(X_future_final_yd)
print("[영덕] 미래 데이터 예측 완료")

[영덕] 미래 데이터 예측 완료


In [12]:
# 6. 제출 파일 생성 (영덕)
pred_df_yd = pd.concat(future_pred_list_yd).sort_values('dt').reset_index(drop=True)
pred_df_yd['predicted_kwh'] = future_predictions_yd
farm_total_pred_yd = pred_df_yd.groupby('dt')['predicted_kwh'].mean().reset_index()
submission_yeongduk = pd.DataFrame({
    'plant_name': '영덕풍력',
    'end_datetime': farm_total_pred_yd['dt'],
    'yield_kwh': farm_total_pred_yd['predicted_kwh']
})
submission_yeongduk['yield_kwh'] = submission_yeongduk['yield_kwh'].clip(lower=0)

submission_yeongduk

,plant_name,end_datetime,yield_kwh
0,영덕풍력,2024-04-01 00:00:00,1564.647583
1,영덕풍력,2024-04-01 01:00:00,2967.648682
2,영덕풍력,2024-04-01 02:00:00,2251.754395
3,영덕풍력,2024-04-01 03:00:00,2120.008057
4,영덕풍력,2024-04-01 04:00:00,2770.488281
...,...,...,...
4339,영덕풍력,2025-02-28 19:00:00,8399.299805
4340,영덕풍력,2025-02-28 20:00:00,9597.816406
4341,영덕풍력,2025-02-28 21:00:00,11180.091797
4342,영덕풍력,2025-02-28 22:00:00,12684.811523


In [13]:
#양양





In [14]:
# 1. 데이터 불러오기 및 기본 전처리 (양양)

try:
    df_train_y_yy = pd.read_parquet("C:/Users/song2/Downloads/jj/train_y_yangyang.parquet")
    df_scada_yy = pd.read_parquet("C:/Users/song2/Downloads/jj/scada_yangyang.parquet")
    print("\n[양양] 데이터 파일 로딩 완료")
except FileNotFoundError:
    print("[양양] 오류: Parquet 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    df_train_y_yy, df_scada_yy = pd.DataFrame(), pd.DataFrame()

if not df_train_y_yy.empty and not df_scada_yy.empty:
    df_train_y_yy['end_datetime'] = pd.to_datetime(df_train_y_yy['end_datetime'])
    df_scada_yy['dt'] = pd.to_datetime(df_scada_yy['dt'])
    df_scada_yy.columns = df_scada_yy.columns.str.strip()
    lower_bound_yy = df_scada_yy['wind_speed_mps'].quantile(0.01)
    upper_bound_yy = df_scada_yy['wind_speed_mps'].quantile(0.99)
    df_scada_yy['wind_speed_mps'] = df_scada_yy['wind_speed_mps'].clip(lower_bound_yy, upper_bound_yy)
    print("[양양] 데이터 기본 전처리 완료")


[양양] 데이터 파일 로딩 완료
[양양] 데이터 기본 전처리 완료


In [15]:
# 2. 피처 엔지니어링 (SCADA 데이터 가공) (양양)

if not df_scada_yy.empty:
    df_scada_yy['month'] = df_scada_yy['dt'].dt.month
    df_scada_yy['hour'] = df_scada_yy['dt'].dt.hour
    df_scada_yy['dayofweek'] = df_scada_yy['dt'].dt.dayofweek
    df_scada_yy['is_weekend'] = df_scada_yy['dayofweek'].isin([5, 6]).astype(int)
    df_scada_yy['hour_sin'] = np.sin(2 * np.pi * df_scada_yy['hour'] / 24)
    df_scada_yy['hour_cos'] = np.cos(2 * np.pi * df_scada_yy['hour'] / 24)
    df_scada_yy['wind_dir_sin'] = np.sin(np.deg2rad(df_scada_yd['wind_direction_degree']))
    df_scada_yy['wind_dir_cos'] = np.cos(np.deg2rad(df_scada_yd['wind_direction_degree']))
    df_scada_yy = df_scada_yy.sort_values(by=['turbine_id', 'dt'])
    for lag in [1, 24]:
        df_scada_yy[f'wind_speed_mps_lag{lag}'] = df_scada_yy.groupby('turbine_id')['wind_speed_mps'].shift(lag)
    rolling_24h_yy = df_scada_yy.groupby('turbine_id')['wind_speed_mps'].rolling(window=24, min_periods=1)
    df_scada_yy['roll_mean_24h'] = rolling_24h_yy.mean().reset_index(level=0, drop=True)
    df_scada_yy['roll_std_24h'] = rolling_24h_yy.std().reset_index(level=0, drop=True)
    df_scada_yy['roll_max_24h'] = rolling_24h_yy.max().reset_index(level=0, drop=True)
    df_scada_yy['roll_min_24h'] = rolling_24h_yy.min().reset_index(level=0, drop=True)
    df_scada_yy['wind_x_hour_sin'] = df_scada_yy['wind_speed_mps'] * df_scada_yy['hour_sin']
    df_scada_yy = pd.get_dummies(df_scada_yy, columns=['turbine_id'], prefix='turbine')
    df_scada_yy = df_scada_yy.interpolate(method='linear').bfill().ffill()
    print("[양양] 피처 엔지니어링 완료")


[양양] 피처 엔지니어링 완료


In [16]:
# 3. 최종 학습 데이터 생성 (양양)

if not df_train_y_yy.empty and not df_scada_yy.empty:
    full_data_yy = pd.merge_asof(
        df_train_y_yy.sort_values("end_datetime"),
        df_scada_yy.sort_values("dt"),
        left_on="end_datetime",
        right_on="dt",
        direction="backward"
    )
    full_data_yy['month'] = full_data_yy['end_datetime'].dt.month
    train_data_yy = full_data_yy[full_data_yy['month'] % 2 == 0].copy()
    features_yy = [col for col in train_data_yy.columns if col not in [
        'end_datetime', 'dt', 'energy_mwh', '구분', '시간', 'energy_kwh',
        'month', 'hour', 'dayofweek'
    ]]
    X_yy = train_data_yy[features_yy]
    y_yy = train_data_yy['energy_kwh']
    y_yy.dropna(inplace=True)
    X_yy = X_yy.loc[y_yy.index]
    print("[양양] 최종 학습 데이터셋 생성 완료")


[양양] 최종 학습 데이터셋 생성 완료


In [17]:
train_data_yy

,end_datetime,구분,시간,energy_mwh,energy_kwh,dt,wind_speed_mps,wind_direction_degree,month,hour,...,turbine_WTG03,turbine_WTG04,turbine_WTG05,turbine_WTG06,turbine_WTG07,turbine_WTG08,turbine_WTG09,turbine_WTG10,turbine_WTG11,turbine_WTG12
0,2024-04-01 00:00:00+09:00,None,None,NaN,NaN,2024-04-01 00:00:00+09:00,6.20,124.89,4,0,...,False,False,False,False,True,False,False,False,False,False
1,2024-04-01 01:00:00+09:00,2024-04-01,1,21.826,21826.0,2024-04-01 01:00:00+09:00,9.05,-143.36,4,1,...,False,False,False,False,False,True,False,False,False,False
2,2024-04-01 02:00:00+09:00,2024-04-01,2,36.470,36470.0,2024-04-01 02:00:00+09:00,10.73,-119.05,4,2,...,False,False,False,False,False,False,True,False,False,False
3,2024-04-01 03:00:00+09:00,2024-04-01,3,36.904,36904.0,2024-04-01 03:00:00+09:00,9.57,-122.38,4,3,...,False,False,False,False,False,False,True,False,False,False
4,2024-04-01 04:00:00+09:00,2024-04-01,4,38.291,38291.0,2024-04-01 04:00:00+09:00,9.35,-129.33,4,4,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4339,2025-02-28 19:00:00+09:00,2025-02-28,19,14.470,14470.0,2025-02-28 19:00:00+09:00,4.84,-114.05,2,19,...,False,False,True,False,False,False,False,False,False,False
4340,2025-02-28 20:00:00+09:00,2025-02-28,20,32.493,32493.0,2025-02-28 20:00:00+09:00,10.76,-104.35,2,20,...,False,False,False,False,False,True,False,False,False,False
4341,2025-02-28 21:00:00+09:00,2025-02-28,21,35.932,35932.0,2025-02-28 21:00:00+09:00,9.76,-119.82,2,21,...,False,True,False,False,False,False,False,False,False,False
4342,2025-02-28 22:00:00+09:00,2025-02-28,22,28.323,28323.0,2025-02-28 22:00:00+09:00,10.56,-107.28,2,22,...,False,False,False,False,False,False,False,False,False,True


In [18]:
# 4. 최종 모델 학습 (양양)

best_params_yy = {
    'lambda': 0.09058223211708799, 
    'alpha': 0.8788050835164622, 
    'colsample_bytree': 0.43691910158013775,
    'subsample': 0.4048448382509827, 
    'learning_rate': 0.010001240733753193, 
    'n_estimators': 643,
    'max_depth': 6, 
    'min_child_weight': 11
}
final_model_yy = xgb.XGBRegressor(random_state=42, n_jobs=-1, **best_params_yy)
final_model_yy.fit(X_yy, y_yy)
print("[양양] 최적 파라미터로 최종 모델 학습 완료")

[양양] 최적 파라미터로 최종 모델 학습 완료


In [19]:
# 5. 미래 데이터 생성 및 예측 (양양)
future_timestamps_yy = pd.to_datetime(pd.date_range(start='2024-04-01', end='2025-03-31', freq='h'))
future_timestamps_yy = future_timestamps_yy[future_timestamps_yy.month.isin([1, 3, 5, 7, 9, 11])]
all_turbines_yy = [col for col in full_data_yy.columns if col.startswith('turbine_WTG')]
X_future_base_yy = pd.DataFrame({'dt': future_timestamps_yy})

# 과거 홀수달의 월/시간별 평균 SCADA 데이터 생성
odd_month_scada_yy = df_scada_yy[df_scada_yy['month'].isin([1, 3, 5, 7, 9, 11])]
historical_averages_yy = odd_month_scada_yy.groupby(['month', 'hour']).agg(
    wind_speed_mps=('wind_speed_mps', 'mean'), wind_direction_degree=('wind_direction_degree', 'mean')
).reset_index()
X_future_base_yy['month'] = X_future_base_yy['dt'].dt.month
X_future_base_yy['hour'] = X_future_base_yy['dt'].dt.hour
X_future_base_yy = pd.merge(X_future_base_yy, historical_averages_yy, on=['month', 'hour'], how='left')
X_future_base_yy['dayofweek'] = X_future_base_yy['dt'].dt.dayofweek
X_future_base_yy['is_weekend'] = X_future_base_yy['dayofweek'].isin([5, 6]).astype(int)
X_future_base_yy['hour_sin'] = np.sin(2 * np.pi * X_future_base_yy['hour'] / 24)
X_future_base_yy['hour_cos'] = np.cos(2 * np.pi * X_future_base_yy['hour'] / 24)
X_future_base_yy['wind_dir_sin'] = np.sin(np.deg2rad(X_future_base_yd['wind_direction_degree']))
X_future_base_yy['wind_dir_cos'] = np.cos(np.deg2rad(X_future_base_yd['wind_direction_degree']))
X_future_base_yy['wind_speed_mps_lag1'] = X_future_base_yy['wind_speed_mps']
X_future_base_yy['wind_speed_mps_lag24'] = X_future_base_yy['wind_speed_mps']
X_future_base_yy['roll_mean_24h'] = X_future_base_yy['wind_speed_mps']
X_future_base_yy['roll_std_24h'] = 0
X_future_base_yy['roll_max_24h'] = X_future_base_yy['wind_speed_mps']
X_future_base_yy['roll_min_24h'] = X_future_base_yy['wind_speed_mps']
X_future_base_yy['wind_x_hour_sin'] = X_future_base_yy['wind_speed_mps'] * X_future_base_yy['hour_sin']
future_pred_list_yy = []
for turbine_col in all_turbines_yy:
    temp_df_yy = X_future_base_yy.copy()
    for col in all_turbines_yy:
        temp_df_yy[col] = (col == turbine_col)
    future_pred_list_yy.append(temp_df_yy)
X_future_final_yy = pd.concat(future_pred_list_yy).sort_values('dt').reset_index(drop=True)
X_future_final_yy = X_future_final_yy.interpolate(method='linear').bfill().ffill()
X_future_final_yy = X_future_final_yy[features_yy]
future_predictions_yy = final_model_yy.predict(X_future_final_yy)
print("[양양] 미래 데이터 예측 완료")


[양양] 미래 데이터 예측 완료


In [20]:
# 6. 제출 파일 생성 (양양)

pred_df_yy = pd.concat(future_pred_list_yy).sort_values('dt').reset_index(drop=True)
pred_df_yy['predicted_kwh'] = future_predictions_yy
farm_total_pred_yy = pred_df_yy.groupby('dt')['predicted_kwh'].mean().reset_index()
submission_yangyang = pd.DataFrame({
    'plant_name': '양양풍력',
    'end_datetime': farm_total_pred_yy['dt'],
    'yield_kwh': farm_total_pred_yy['predicted_kwh']
})
submission_yangyang['yield_kwh'] = submission_yangyang['yield_kwh'].clip(lower=0)

submission_yangyang

,plant_name,end_datetime,yield_kwh
0,양양풍력,2024-05-01 00:00:00,12669.055664
1,양양풍력,2024-05-01 01:00:00,12128.352539
2,양양풍력,2024-05-01 02:00:00,11712.416016
3,양양풍력,2024-05-01 03:00:00,11217.547852
4,양양풍력,2024-05-01 04:00:00,12610.205078
...,...,...,...
4388,양양풍력,2025-03-30 20:00:00,10303.306641
4389,양양풍력,2025-03-30 21:00:00,11241.063477
4390,양양풍력,2025-03-30 22:00:00,12388.721680
4391,양양풍력,2025-03-30 23:00:00,12320.084961


In [ ]:
# 1. 영덕과 양양의 예측 결과를 하나의 데이터프레임으로 합치기
final_submission = pd.concat([submission_yeongduk, submission_yangyang], ignore_index=True)

# 2. 발전소 이름(영덕 -> 양양 순)과 시간 순서로 정렬
final_submission = final_submission.sort_values(
 by=['plant_name', 'end_datetime'],
    ascending=[False, True]  
).reset_index(drop=True)

# 3. 최종 결합 결과 확인 (상위 5개와 하위 5개)
print("======= 최종 결합 결과 (상위 5개) =======")
print(final_submission.head())
print("\n======= 최종 결합 결과 (하위 5개) =======")
print(final_submission.tail())


# 4. CSV 파일로 저장 
final_submission.to_csv("submission.csv", index=False, encoding='utf-8-sig')
print("\n'submission.csv' 파일 저장이 완료되었습니다.")

======= 최종 결합 결과 (상위 5개) =======
  plant_name        end_datetime    yield_kwh
0       영덕풍력 2024-04-01 00:00:00  1564.647583
1       영덕풍력 2024-04-01 01:00:00  2967.648682
2       영덕풍력 2024-04-01 02:00:00  2251.754395
3       영덕풍력 2024-04-01 03:00:00  2120.008057
4       영덕풍력 2024-04-01 04:00:00  2770.488281

======= 최종 결합 결과 (하위 5개) =======
     plant_name        end_datetime     yield_kwh
8732       양양풍력 2025-03-30 20:00:00  10303.306641
8733       양양풍력 2025-03-30 21:00:00  11241.063477
8734       양양풍력 2025-03-30 22:00:00  12388.721680
8735       양양풍력 2025-03-30 23:00:00  12320.084961
8736       양양풍력 2025-03-31 00:00:00  11604.008789

'submission.csv' 파일 저장이 완료되었습니다.
